In [1]:
import spotipy 
import pandas as pd 
from spotipy.oauth2 import SpotifyClientCredentials
import os
import numpy as np

import numpy as np
import pandas as pd
import pickle
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from matplotlib import pyplot
from sklearn.metrics import silhouette_score
from IPython.display import IFrame

<a id="functions"></a>
## 1. Functions

In [2]:
def load_songs(fp:[str]=None):
    if not fp:
        print("No filepath provided")
        return None
    try:
        return pd.read_csv(fp, sep=";", error_bad_lines=False)
    except FileNotFoundError:
        print("File not found.")
        return None

In [3]:
def load_pickle(fp:[str] = None): 
    if not fp:
        print("no filepath provided.")
        return None
    try: 
        with open(fp, "rb") as f: 
            return pickle.load(f) 
    except FileNotFoundError: 
        print("File not found!")
        return None

In [17]:
def user_input_get_features_old(this_df):
    audio_feat = ['danceability', 'energy', 'key', 'loudness', 'speechiness', 'acousticness', 'instrumentalness',
                  'liveness', 'valence', 'tempo', 'time_signature']
    user_input = input("What song are you looking for? ")
    song_hits = list(this_df[this_df["name"].str.lower()==user_input.lower()]["artist"])
    if song_hits:
        if len(song_hits) == 1:
            numpy_lst = []
            for a_f in audio_feat:
                numpy_lst.append(float(this_df[this_df["name"].str.lower()==user_input.lower()][a_f]))
            return np.array(numpy_lst).reshape(1,-1)
        else:
            print(f'Following artists found for "{user_input}":\n ')
            c = 0
            for a in song_hits:
                c += 1
                print(f" + {c}: {a}")
            print(" -------\n+ S: Search alternative on Spotify")
            user_choice = 241
            while 0 > int(user_choice) or int(user_choice) > len(song_hits)+1:
                user_choice = input(f"Please specify your entry (1 to {len(song_hits)}): ")
                if user_choice.lower() == "s":
                    return get_unknown_song_array(user_input)
                if user_choice.isnumeric() == False:
                    print("please check entry.")
                    user_choice = len(song_hits)+2
                if 0 > int(user_choice) > len(song_hits)+1:
                    print("please check entry.")
            this_df_index = list(this_df[this_df["name"].str.lower()==user_input.lower()].index)[int(user_choice)-1]
            # print(df_.iloc[df_index]["danceability"])
            numpy_lst = []
            for a_f in audio_feat:
                numpy_lst.append(float(this_df.iloc[this_df_index][a_f]))
            return np.array(numpy_lst).reshape(1,-1)
    else:
        print("Song not found in local database, searching on Spotify.")
        return get_unknown_song_array(user_input)

In [18]:
def user_input_get_features(this_df):
    audio_feat = ['danceability', 'energy', 'key', 'loudness', 'speechiness', 'acousticness', 'instrumentalness',
                  'liveness', 'valence', 'tempo', 'time_signature']
    user_input = input("What song are you looking for? ")
    song_hits = list(this_df[this_df["name"].str.lower()==user_input.lower()]["artist"])
    if song_hits:
        if len(song_hits) == 1:
#             numpy_lst = []
#             for a_f in audio_feat:
#                 numpy_lst.append(float(this_df[this_df["name"].str.lower()==user_input.lower()][a_f]))
#             return np.array(numpy_lst).reshape(1,-1)
            print(f'Following artist found for "{user_input}":\n ')
            print(f' + 1: {song_hits[0]}')
            print(" -------\n+ S: Search alternative on Spotify")
        else:
            print(f'Following artists found for "{user_input}":\n ')
            c = 0
            for a in song_hits:
                c += 1
                print(f" + {c}: {a}")
            print(" -------\n+ S: Search alternative on Spotify")
        user_choice = 241
        while 0 > int(user_choice) or int(user_choice) > len(song_hits)+1:
            user_choice = input(f"Please specify your entry (1 to {len(song_hits)}): ")
            if user_choice.lower() == "s":
                return get_unknown_song_array(user_input)
            if user_choice.isnumeric() == False:
                print("please check entry.")
                user_choice = len(song_hits)+2
            if 0 > int(user_choice) > len(song_hits)+1:
                print("please check entry.")
        this_df_index = list(this_df[this_df["name"].str.lower()==user_input.lower()].index)[int(user_choice)-1]
        # print(df_.iloc[df_index]["danceability"])
        numpy_lst = []
        for a_f in audio_feat:
            numpy_lst.append(float(this_df.iloc[this_df_index][a_f]))
        return np.array(numpy_lst).reshape(1,-1)
    else:
        print("Song not found in local database, searching on Spotify.")
        return get_unknown_song_array(user_input)

In [5]:
def get_unknown_song_array(this_song:[str]=None):
    if not this_song:
        print("no song provided.")
        return None
    x=sp.search(this_song, limit=15)
    if not x["tracks"]["items"]:
        print("song is not on spotify.")
        return None
    if len(x["tracks"]["items"]) == 1:
        pass
    else:
        artists_of_this_song = []
        for _ in range(len(x["tracks"]["items"])):
            artists_of_this_song.append(x["tracks"]["items"][_]["artists"][0]["name"])
        # artists_of_this_song = sorted(set(artists_of_this_song), key=lambda x: x.lower())
        print(f'Following artists found for "{this_song}":\n ')
        c = 0
        for a in artists_of_this_song:
            c += 1
            print(f" + {c}: {a}")
        user_choice = 241
        while 0 > int(user_choice) or int(user_choice) > len(artists_of_this_song) + 1:
            user_choice = input(f"Please specify your entry (1 to {len(artists_of_this_song)}): ")
            if user_choice.isnumeric() == False:
                print("please check entry.")
                user_choice = len(artists_of_this_song) + 2   # +2 to create a wrong int and run another while loop 
            if 0 > int(user_choice) > len(song_hits) + 1:
                print("please check entry.")
        # print((x["tracks"]["items"][int(user_choice)-1]["artists"][0]["name"]))
        song_info=sp.audio_features([x["tracks"]["items"][int(user_choice)-1]["id"]])
        remove_list = ['mode', 'duration_ms', 'id', 'type', 'uri', 'track_href', 'analysis_url'] #remove features not used in the model
        [song_info[0].pop(key) for key in remove_list] 
        song_list=list(song_info[0].values())
        song_list
        song_array = np.array(song_list).reshape(1, -1)
        song_array
        return song_array

In [23]:
def get_random_song_cluster(this_df, this_song_array):
    X = song_df.drop(['id', 'length','name', 'album','artist','release_date', 'type', 'mode', 'popularity', "cluster"], axis=1) 
    scaler = load_pickle("Model/scaler.pickle")
    kmeans = load_pickle("Model/kmeans_10.pickle")
    scaled_song = scaler.transform(this_song_array)
    cluster_pred = kmeans.predict(scaled_song)[0]
    print("cluster:", cluster_pred)
    index_song = this_df[this_df['cluster'] == cluster_pred].sample().index[0]
    id_song = this_df.iloc[index_song]['id']
    return print(f"{sp.track(id_song)['name']} by {sp.track(id_song)['artists'][0]['name']} (Spotify ID: {id_song})")

In [7]:
def add_cluster_column_to_song_df(this_df):
    #define features
    X = this_df.drop(['id', 'length','name', 'album','artist','release_date', 'type', 'mode', 'popularity'], axis=1) 
    
    scaler = StandardScaler()
    scaler.fit(X)
    X_scaled = scaler.transform(X)
    X_scaled_df = pd.DataFrame(X_scaled, columns = X.columns)
    
    kmeans = KMeans(n_clusters=10, random_state=42)
    kmeans.fit(X_scaled_df)
    
    clusters = kmeans.predict(X_scaled_df)
    #clusters
    pd.Series(clusters).value_counts().sort_index()
    this_df["cluster"] = clusters
    return this_df

In [53]:
# song_df

In [9]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="9d897a87744a49638d588daeca7f0994",
                                                           client_secret="1e0c19e2db6c4e3182c30885a6491ffd"))

In [20]:
song_df = load_songs("./data/song_db.csv")

C:\Users\roman\AppData\Local\Temp/ipykernel_10952/701837643.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  song_df = load_songs("./data/song_db.csv")
b'Skipping line 3335: expected 20 fields, saw 21\nSkipping line 3336: expected 20 fields, saw 21\nSkipping line 3343: expected 20 fields, saw 21\nSkipping line 3345: expected 20 fields, saw 21\nSkipping line 3350: expected 20 fields, saw 21\nSkipping line 4630: expected 20 fields, saw 22\nSkipping line 4632: expected 20 fields, saw 22\nSkipping line 4634: expected 20 fields, saw 22\nSkipping line 4643: expected 20 fields, saw 22\nSkipping line 4644: expected 20 fields, saw 22\nSkipping line 4645: expected 20 fields, saw 22\nSkipping line 4646: expected 20 fields, saw 22\nSkipping line 4647: expected 20 fields, saw 22\nSkipping line 4650: expected 20 fields, saw 22\nSkipping line 4651: expected 20 fields, saw 22\nSkipping line 4655: expected 20 fields, saw 22\nSkipping lin

In [21]:
song_df = add_cluster_column_to_song_df(song_df)

In [28]:
song_array = user_input_get_features(song_df)
get_random_song_cluster(song_df, song_array)

What song are you looking for? Where Is The Love
Following artists found for "Where Is The Love":
 
 + 1: Betty Wright
 + 2: Donny Hathaway
 -------
+ S: Search alternative on Spotify
Please specify your entry (1 to 2): 1
cluster: 2
Bite Me - Acoustic by Avril Lavigne (Spotify ID: 71BPElgdb5MtfIJYGuTWqB)


C:\Users\roman\Python\anaconda\envs\DA_Env\lib\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(


In [ ]:
# track_id = "4yzlbUDqrP1ZriRp7zAcIP"
# IFrame(src="https://open.spotify.com/embed/track/"+track_id,
#        width="320",
#        height="80",
#        frameborder="0",
#        allowtransparency="true",
#        allow="encrypted-media",
#       )